# 单历元定位要求基于GPS系统的C1C和C2W观测值，组成无电离层组合的观测值，计算2019年7月15号06:55:30的坐标。
## hkws1960.19o是观测文件，igs20621.sp3是精密星历文件。

##### 1.读取hkws1960.19o观测文件，将GPS系统的C1C和C2W观测值分别存在两个列表中。

In [1]:
import numpy as np

obsfile = 'hkws1960.19o'
satnames = [] # 保存卫星编号
c1c = [] # 保存L1伪距观测值
c2w = [] # 保存L2伪距观测值

with open(obsfile) as file_object:
    for line in file_object:
        if line[2:29] == '2019 07 15 06 55 30.0000000':
            number = int(line[32:35])
            for i in range(number):
                if line[0] == 'G':  # 选出所有GPS观测数据
                    # 每个数据占16列，第i个数据的起始列数为4 + 16(𝑖 − 1)
                    satnames.append(line[0:3])
                    c1c.append(float(line[3+16*(1-1):4+16*(1-1)+16]))  # C1C为第1个数据
                    c2w.append(float(line[3+16*(9-1)-1:4+16*(9-1)+16]))  #C2W为第9个数据
                line = file_object.readline() # 读取行不以字符'G'开头，继续向下读取
            break # 收集完'2019 07 15 06 55 30.0000000'时的数据即可退出文件读取循环

if len(c1c)==len(c2w):
    satnum = len(c1c) 
    
p1 = np.array(c1c) #存在1*9的矩阵中
p2 = np.array(c2w)

In [2]:
satnames
satnum

9

##### 2.读取igs20621.sp3精密星历文件，获得卫星所在星历的时间、位置和钟差信息

In [3]:
igsfile = 'igs20621.sp3'
# 需定位的时间为2019年7月15号06:55:30 
# 根据SP3文件每15min一组数据（起始时刻为00:00:00），在需定位时间之前最接近的星历时间为2019年7月15号06:45:00 
# 从这个时间点再向前推4组，即选取2019年7月15号05:45：00 
igsT = [] # 存储观测卫星在7月15各个所取星历的时间 (min)
sXdata = [] # 存储观测卫星在各个所取星历时间的x坐标 (km)
sYdata = [] # 存储观测卫星在各个所取星历时间的y坐标 (km)
sZdata = [] # 存储观测卫星在各个所取星历时间的z坐标 (km)
sClock = [] # 存储观测卫星在各个所取星历时间的钟差(μs)

with open(igsfile) as file_object:
    for line in file_object:
        if line[3:31] == '2019  7 15  5 45  0.00000000':
            igsT.append(float(line[14:16])*60 + float(line[17:19]) + float(line[20:31])/60) 
            
            # 卫星位置（km）和钟差（μs）依次为X、Y、Z、钟差每个数据占14列
            # 第i个数据的起始列为 5 + 14(i - 1)
            while(len(igsT)<=10):   # 以时间t为准，向前找5组精密星历，向后找5组精密星历，需10组
                line = file_object.readline()
                for n in range(satnum):  # 读一组选取时间点的星历，存储9个观测卫星参数
                    while line[1:4] != satnames[n]:
                        line = file_object.readline()
                    sXdata.append(float(line[4+14*(1-1):5+14*(1-1)+14]))                            
                    sYdata.append(float(line[4+14*(2-1):5+14*(2-1)+14]))
                    sZdata.append(float(line[4+14*(3-1):5+14*(3-1)+14]))
                    sClock.append(float(line[4+14*(4-1):5+14*(4-1)+14]))
                if len(igsT) == 10:
                    break
                else:
                    while(line[0] != '*'):
                        line = file_object.readline()
                    igsT.append(float(line[14:16])*60 + float(line[17:19]) + float(line[20:31])/60) 
            break

            

In [4]:
igsT
len(sXdata)

90

In [5]:
sXmtr = np.array(sXdata).reshape(satnum,10)
sYmtr = np.array(sYdata).reshape(satnum,10)
sZmtr = np.array(sZdata).reshape(satnum,10)
sCmtr = np.array(sClock).reshape(satnum,10)

In [15]:
sXmtr

array([[ 13759.577218,   5443.613066,   1508.501602, -24466.853703,
         14162.656881, -11750.975259,  -5164.670858, -25884.150666,
        -12915.242229,  13016.695402],
       [  4701.165208,     85.414037, -25036.802625,  13164.618072,
        -13810.346006,  -7611.410737, -25040.669237, -12905.582943,
         12045.355038,   4099.744755],
       [ -1534.38156 , -25308.376379,  12302.234942, -15669.765144,
         -9929.313823, -23952.094217, -12810.456866,  10841.004578,
          3619.36445 ,  -3332.746151],
       [-25287.022798,  11588.77204 , -17296.861402, -12073.870973,
        -22648.928284, -12669.323964,   9406.919266,   3234.063549,
         -5283.641597, -24986.564807],
       [ 11028.910758, -18666.394787, -14006.564767, -21166.709316,
        -12521.854866,   7754.179649,   2913.08721 ,  -7353.953004,
        -24428.617437,  10618.699401],
       [-19761.107258, -15696.174568, -19544.753388, -12406.197033,
          5901.380266,   2622.267675,  -9504.586309, -236

##### 3.以igsT时间为自变量使用9阶拉格朗日插值，解算出所求时刻的观测卫星信息

In [7]:
from scipy.interpolate import lagrange

xs = []
ys = []
zs = []
dts = []
t = 6*60 + 55 + 30/60  # t为插值时间

for i in range(satnum):
    sx_poly = lagrange(igsT, sXmtr[i])
    xs.append(sx_poly(t))
    sy_poly = lagrange(igsT, sYmtr[i])
    ys.append(sy_poly(t))
    sz_poly = lagrange(igsT, sZmtr[i])
    zs.append(sz_poly(t))
    sc_poly = lagrange(igsT, sCmtr[i])
    dts.append(sc_poly(t))

# 将观测卫星的位置和钟差转换为矩阵
satPos = np.array([xs, ys, zs])
dTs = np.array(dts)

xs = np.array(xs)
ys = np.array(ys)
zs = np.array(zs)
xs *= 1000
ys *= 1000 
zs *= 1000

satPos *= 1000 # km转换为m
dTs *= 10**(-6) # μs转换为s

In [8]:
satPos

array([[ -2354937.5       , -10352578.125     , -19325945.3125    ,
        -18843910.15625   ,   3284648.4375    ,   5411640.625     ,
         -6137527.34375   , -24541828.125     ,   4927609.375     ],
       [ 10787922.42050171,  17398281.25      ,    366976.5625    ,
         13347710.9375    ,  19989890.625     ,  24647195.3125    ,
         16643605.46875   ,   2140833.984375  ,  17772179.6875    ],
       [ 22415593.75      ,  19450652.34375   ,  17239257.8125    ,
         -7119328.125     ,  10296125.        ,   4590625.        ,
         14267890.625     ,  18341359.375     ,  12615777.34375   ]])

In [9]:
dTs

array([ 1.47648376e-04, -1.75914989e-04, -3.58944092e-04,  6.30683105e-04,
        6.77539062e-05, -1.77857422e-04,  1.28061523e-04,  3.22950195e-04,
        2.83077881e-04])

##### 4.组成观测方程(V=L-A*dX, P)，P可取单位阵

In [10]:
from math import sqrt
from numpy.linalg import inv

f1 = 1575.42 # C1C载波频率
f2 = 1227.6 # C2W载波频率
c = 299792458
PC = f1**2/(f1**2 - f2**2)*p1 - f2**2/(f1**2 - f2**2)*p2
dX = np.ones(4)
X = np.zeros(4)
re = []

while sqrt(dX[0]**2+dX[1]**2+dX[2]**2) >= 0.0001:
    Li = []
    Ai = []
    for i in range(satnum):
        pho = sqrt((satPos[0,i]-X[0])**2+(satPos[1,i]-X[1])**2+(satPos[2,i]-X[2])**2)
        Li.append(PC[i]-pho-c*X[3]+c*dTs[i])
        Ai.append([(X[0]-satPos[0,i])/pho, (X[1]-satPos[1,i])/pho, (X[2]-satPos[2,i])/pho, c])
    A = np.array(Ai).reshape(satnum, 4)
    L = np.array(Li)
    Q = inv(A.T.dot(A))
    dX = Q.dot(A.T.dot(L))
    re.append(dX)
    X += dX



In [11]:
X

array([-4.14271770e+06,  1.77850590e+06,  2.47518704e+06,  1.00993203e-03])

In [12]:
p1

dX = np.ones(4)
dX
L

array([  715364.44827855, -1737510.21973805,  -117082.78819306,
        1487298.21896209,  1699692.03902309, -4180190.64951654,
        1288063.10881006, -1003434.17954668,  1847800.02192053])

In [13]:
a1 = np.array([1, 2, 3]) 
a2 = np.array([4, 5, 6])
#a = np.append(a2, a1)
P = np.identity(3)
a2 * a1
a3 = np.array([a1, a2])
a4 = np.array([a1, a2]).T
inv(a4.T.dot(P).dot(a4))
2*inv(np.array([[1,0],[0,1]]))
a1.T.dot(a2)

32

In [14]:
a=np.array([[1,2,3],[4,5,6]])
b=np.array([[11,21,31],[7,8,9]])
a.shape
b.shape
np.concatenate((a,b),axis=1)
c = np.array()
c = np.concatenate(c, a)

TypeError: array() missing required argument 'object' (pos 1)

In [ ]:
a1 = np.array([[1, 2, 3],[2, 3, 4]]) 
a2 = np.array([[4, 5, 6],[5, 6, 7]])
inv(a1.dot(a2.T))

In [ ]:
a1 = np.array([[1, 2, 3],[1, 2, 3]]) 
a2 = np.array([4, 5, 6])
a2.dot(a1.T)